In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

# new version

In [2]:
p=2 #nbr of classes
k=256 #dimension of the dictionary
n=100 # dimension of the signal

#random variables
alpha_0 = np.random.randn(k)
w = np.random.randn(k, 1)
b = np.random.randn(1)
l = 1
D = np.random.randn(n, k)
signals = np.random.randn(10, n)
lambda_0 = 1 #see later how to choose this

mu =np.linspace(0, 1, 10)


In [3]:
# In the case of binary classification

def function_S(alpha, x, D, w, b, lambda_0, lambda_1, pos=True, method="linear"):
    if method == "linear":
        if pos:
            logistic_loss = np.log(1 + np.exp(-(w.T @ alpha + b)))
        else:
            logistic_loss = np.log(1 + np.exp(w.T @ alpha + b))
    else:
        print("Method not implemented")
        return np.nan

    reg_1 = lambda_0 * np.linalg.norm(x - D @ alpha, ord=2)**2
    reg_2 = lambda_1 * np.linalg.norm(alpha, ord=1)

    return logistic_loss + reg_1 + reg_2


In [4]:

def supervised_sparse_coding(alpha, signals, D, w, b):
    """Supervised sparse coding step for 2 classes"""

    n = signals.shape[0] #nbr of signals
    size_dict = D.shape[1] #dictionary size
    lambda_0 = 1
    lambda_1 = 0.15

    alpha_opt_pos = np.zeros((n, size_dict))
    alpha_opt_neg = np.zeros((n, size_dict))

    alpha_pos = alpha
    alpha_neg = alpha

    for i in range(n):
        print(i)
        x = signals[i, :]
        # optimize on alpha S_neg(-1 f(alpha, x_i, D, theta)
        alpha_temp_neg = scipy.optimize.minimize(function_S, alpha_neg, args= (x, D, w, b, lambda_0, lambda_1, False), tol=1e-6)
        # optimize on alpha S_neg(1 f(alpha, x_i, D, theta)
        alpha_temp_pos = scipy.optimize.minimize(function_S, alpha_pos, args= (x, D, w, b, lambda_0, lambda_1, True), tol=1e-6)

        alpha_opt_pos[i,:] = alpha_temp_pos.x
        alpha_opt_neg[i,:] = alpha_temp_neg.x
        alpha_pos = alpha_temp_pos.x
        alpha_neg = alpha_temp_neg.x

    return alpha_opt_neg, alpha_opt_pos


In [5]:
alpha_opt_neg, alpha_opt_pos = supervised_sparse_coding(alpha_0, signals, D, w, b)

0
1


KeyboardInterrupt: 

In [ ]:
plt.plot([np.linalg.norm(alpha_opt_pos[i] - alpha_opt_pos[-1], ord=1) for i in range(alpha_opt_pos.shape[0])])
plt.show()

In [6]:
def gradC(x):
    # gradient of the logistic loss
    return -1/(1+np.exp(x))

def compute_gradients(D, w, b, X, Y, alpha, lambda_0, mu):
    """Compute gradients of E with respect to D, w, and b."""
    # Initialize gradients
    grad_D = np.zeros_like(D)
    grad_w = np.zeros_like(w)
    grad_b = 0.0

    # Iterate over data samples
    for i in range(len(X)):
        for z in [-1, 1]: #0 = class -1, 1 = class 1
            # Compute residual and coefficients
            if z==-1:
                p = 0
            else:
                p = 1
            alpha_i_z = alpha[:, i, p]  # Sparse code
            residual = X[i] - D @ alpha_i_z

            # Compute omega_i_z (gradient of C with respect to w^T alpha + b)
            wx_plus_b = w.T @ alpha_i_z + b
            diff_S = function_S(alpha[:, i, 0], X[i], D, w, b, lambda_0, pos=(Y[i]))
            grad_C = (-1 / (1 + np.exp((z * wx_plus_b)))) * z

            omega_i_z = -mu * grad_C

            # Gradients
            grad_D -= 2 * lambda_0 * omega_i_z * np.outer(residual, alpha_i_z)
            grad_w += omega_i_z * alpha_i_z
            grad_b += omega_i_z

        return grad_D, grad_w, grad_b

def project_D(D):
    """Project dictionary D to satisfy column constraints (e.g., unit-norm)."""
    return D / np.maximum(np.linalg.norm(D, axis=0, keepdims=True), 1e-8)

In [7]:
def projected_gradient_descent(D, w, b, signals, alpha, lambda_0, mu, grad_steps=100, step_size=0.01, tol=1e-6):
    """
    Projected Gradient Descent for optimizing E(D, theta) with dictionary D and linear parameters w, b.

    Parameters:
        D (ndarray): Initial dictionary, shape (d, k)
        w (ndarray): Initial linear weights, shape (k,)
        b (float): Initial bias term
        X (ndarray): Input data matrix, shape (m, d)
        alpha (ndarray): Sparse codes, shape (k, m, 2) for each z={-1,+1}
        lambda_0 (float): Regularization parameter
        mu (float): Classification tradeoff parameter
        grad_steps (int): Number of gradient descent iterations
        step_size (float): Step size for updates
        tol (float): Tolerance for convergence

    Returns:
        D, w, b: Updated parameters
    """

    m, d = signals.shape  # m samples, d dimensions
    k = D.shape[1]  # k dictionary atoms

    def compute_gradients(D, w, b, signals, alpha, lambda_0, mu):
        """Compute gradients of E with respect to D, w, and b."""
        # Initialize gradients
        grad_D = np.zeros_like(D)
        grad_w = np.zeros_like(w)
        grad_b = 0.0

        # Iterate over data samples
        for i in range(m):
            for z in [-1, +1]:
                # Compute residual and coefficients
                alpha_i_z = alpha[:, i, z]
                residual = signals[i] - D @ alpha_i_z

                # Compute omega_i_z
                grad_C = (w.T @ alpha_i_z + b)  # Assume a placeholder gradient of C
                omega_i_z = -mu * z * grad_C

                # Gradients
                grad_D -= 2 * lambda_0 * omega_i_z * np.outer(residual, alpha_i_z)
                grad_w += omega_i_z * z * grad_C * alpha_i_z
                grad_b += omega_i_z * z * grad_C

        return grad_D, grad_w, grad_b


    # Gradient Descent Loop
    for step in range(grad_steps):
        # Compute gradients
        grad_D, grad_w, grad_b = compute_gradients(D, w, b, X, alpha, lambda_0, mu)

        # Update parameters
        D -= step_size * grad_D
        w -= step_size * grad_w
        b -= step_size * grad_b

        # Project dictionary D
        D = project_D(D)

        # Convergence Check (based on gradient norms)
        if np.linalg.norm(grad_D) < tol and np.linalg.norm(grad_w) < tol and abs(grad_b) < tol:
            print(f"Converged at step {step + 1}")
            break

    return D, w, b


In [24]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from scipy.optimize import minimize
from joblib import Parallel, delayed

class SupervisedDictionaryLearning(BaseEstimator, ClassifierMixin):
    def __init__(self, n_components=10, lambda0=0.1, lambda1=0.1, lambda2=0.1, max_iter=100, tol=1e-4, n_jobs=-1):
        """
        Parameters:
        - n_components: int, size of the dictionary (k)
        - lambda0, lambda1, lambda2: float, regularization parameters
        - max_iter: int, maximum number of iterations
        - tol: float, convergence tolerance
        - n_jobs: int, number of parallel jobs to run for sparse coding (default is -1 for all CPUs)
        """
        self.n_components = n_components
        self.lambda0 = lambda0
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.max_iter = max_iter
        self.tol = tol
        self.n_jobs = n_jobs


    def _initialize(self, X):
        n_features = X.shape[1]
        self.D_ = np.random.randn(n_features, self.n_components)
        self.D_ /= np.linalg.norm(self.D_, axis=0)  # Normalize columns
        self.theta_ = np.zeros(self.n_components + 1)  # For linear model (w, b)

    def _sparse_coding(self, X):
        """Solve supervised sparse coding for each sample in parallel."""
        m = X.shape[0]
        # Parallelize the computation of alpha for each sample
        alpha_neg = Parallel(n_jobs=self.n_jobs)(delayed(self._solve_alpha)(X[i], -1) for i in range(m))
        alpha_pos = Parallel(n_jobs=self.n_jobs)(delayed(self._solve_alpha)(X[i], 1) for i in range(m))
        return np.array(alpha_neg), np.array(alpha_pos)

    def _solve_alpha(self, xi, label):
        """Solve the sparse coding problem for a single sample."""
        def objective(alpha):
            reconstruction_error = np.linalg.norm(xi - self.D_ @ alpha)**2
            sparsity_penalty = self.lambda1 * np.linalg.norm(alpha, 1)
            classification_loss = np.log(1 + np.exp(-label * (self.theta_[:-1] @ alpha + self.theta_[-1])))
            return classification_loss + self.lambda0 * reconstruction_error + sparsity_penalty

        result = minimize(objective, np.zeros(self.n_components), method='L-BFGS-B')
        return result.x

    def _update_dictionary_and_params(self, X, y, alpha_neg, alpha_pos, lr_D=0.01, lr_theta=0.01):
        """Update D and theta using provided gradients."""
        m = X.shape[0]
        self.list_mu = 0*np.linspace(0, 1, m)
        grad_D = np.zeros_like(self.D_)
        grad_theta_w = np.zeros(self.n_components)
        grad_theta_b = 0

        for i in range(m):
            xi = X[i]
            yi = y[i]

            # Compute omega
            S_neg = self._compute_S(alpha_neg[i], xi, yi, -1)
            S_pos = self._compute_S(alpha_pos[i], xi, yi, +1)
            omega_neg = - self.list_mu[i] * (-1) * self._nabla_C(S_neg - S_pos) + (1 - self.list_mu[i]) * int(yi == -1)
            omega_pos = -self.list_mu[i] * (+1) * self._nabla_C(S_neg - S_pos) + (1 - self.list_mu[i]) * int(yi == +1)

            # Update gradients for D
            grad_D += omega_neg * np.outer((xi - self.D_ @ alpha_neg[i]), alpha_neg[i])
            grad_D += omega_pos * np.outer((xi - self.D_ @ alpha_pos[i]), alpha_pos[i])

            # Update gradients for theta (w and b)
            pred_neg = self.theta_[:-1] @ alpha_neg[i] + self.theta_[-1]
            pred_pos = self.theta_[:-1] @ alpha_pos[i] + self.theta_[-1]
            grad_theta_w += omega_neg * (-1) * self._nabla_C(pred_neg) * alpha_neg[i]
            grad_theta_w += omega_pos * (+1) * self._nabla_C(pred_pos) * alpha_pos[i]
            grad_theta_b += omega_neg * (-1) * self._nabla_C(pred_neg)
            grad_theta_b += omega_pos * (+1) * self._nabla_C(pred_pos)

        grad_D *= - 2 * self.lambda0
        # Gradient updates
        self.D_ -= lr_D * grad_D  # Learning rate for D
        self.D_ /= np.linalg.norm(self.D_, axis=0)  # Re-normalize columns
        self.theta_[:-1] -= lr_theta * grad_theta_w  # Learning rate for w
        self.theta_[-1] -= lr_theta * grad_theta_b  # Learning rate for b

    def _compute_S(self, alpha, xi, yi, label):
        """Compute the loss S for a given alpha."""
        reconstruction_error = np.linalg.norm(xi - self.D_ @ alpha)**2
        sparsity_penalty = self.lambda1 * np.linalg.norm(alpha, 1)
        classification_loss = np.log(1 + np.exp(-label * (self.theta_[:-1] @ alpha + self.theta_[-1])))
        return classification_loss + self.lambda0 * reconstruction_error + sparsity_penalty

    def _nabla_C(self, x):
        """Gradient of the logistic loss."""
        return -1 / (1 + np.exp(x))

    def fit(self, X, y):
        """Fit the model to the data."""
        self._initialize(X)
        for iteration in range(self.max_iter):
            alpha_neg, alpha_pos = self._sparse_coding(X)
            self._update_dictionary_and_params(X, y, alpha_neg, alpha_pos)

            # Log progress
            current_loss = np.mean([self._compute_S(alpha_neg[i], X[i], y[i], -1) +
                                    self._compute_S(alpha_pos[i], X[i], y[i], +1) for i in range(len(y))])
            print(f"Iteration {iteration + 1}/{self.max_iter}, Loss: {current_loss:.4f}")

            # Check convergence
            if iteration > 0 and np.linalg.norm(alpha_neg - alpha_pos) < self.tol:
                print("Convergence reached.")
                break

        return self

    def predict(self, X):
        """Predict class labels for samples in X."""
        alpha_list = np.array([self._solve_alpha(xi, 1) for xi in X])  # Assume yi=1 for prediction
        scores = alpha_list @ self.theta_[:-1] + self.theta_[-1]
        return np.sign(scores)

    def score(self, X, y):
        """Return the classification accuracy."""
        y_pred = self.predict(X)
        return np.mean(y_pred == y)


In [12]:
class SyntheticTimeSeriesDataset:
    def __init__(self, num_classes=3, num_samples_per_class=100,
                 sequence_length=100):
        self.num_classes = num_classes
        self.num_samples_per_class = num_samples_per_class
        self.sequence_length = sequence_length
        self.data = None
        self.labels = None

    def generate_class_data(self, class_id):
        np.random.seed(class_id)
        data, labels = [], []
        time = np.linspace(0, 2 * np.pi, self.sequence_length)

        for _ in range(self.num_samples_per_class):
            if class_id == 0:
                signal = (
                    np.sin((class_id + 1) * time) +
                    0.1 * np.random.randn(self.sequence_length)
                )
            elif class_id == 1:
                signal = (
                    np.cos((class_id + 1) * time) +
                    0.1 * np.random.randn(self.sequence_length)
                )
            elif class_id == 2:
                signal = (
                    np.sin((class_id + 1) * time) *
                    np.exp(-0.05 * time) +
                    0.1 * np.random.randn(self.sequence_length)
                )
            data.append(signal)
            labels.append(class_id)
        return np.array(data), np.array(labels)

    def create_dataset(self):
        data, labels = [], []
        for class_id in range(self.num_classes):
            class_data, class_labels = self.generate_class_data(class_id)
            data.append(class_data)
            labels.append(class_labels)
        self.data, self.labels = np.vstack(data), np.hstack(labels)
        return self.data, self.labels  # Explicitly return X and y



    def plot_examples(self):
        if self.data is None or self.labels is None:
            raise ValueError("Dataset has not been created. "
                             "Call create_dataset() first.")
        plt.figure(figsize=(10, 6))
        for class_id in range(self.num_classes):
            idx = class_id * self.num_samples_per_class
            plt.plot(self.data[idx], label=f"Class {class_id}")
        plt.title("Example Time Series for Each Class")
        plt.xlabel("Time Step")
        plt.ylabel("Value")
        plt.legend()
        plt.show()


In [27]:
# Generate the entire dataset
from sklearn.model_selection import train_test_split
dataset = SyntheticTimeSeriesDataset(num_classes=2, num_samples_per_class=100, sequence_length=100)
X, y = dataset.create_dataset()

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Use the train set to fit the model
# choice of parameters so the lambda1/lambda0 ratio is 0.15
model = SupervisedDictionaryLearning(n_components=10, lambda0=1, lambda1=0.15, lambda2=0.2, max_iter=1)
model.fit(X_train, y_train)

# Evaluate the model on the test set
accuracy = model.score(X_test, y_test)
print("Test set accuracy:", accuracy)


Iteration 1/1, Loss: 57.9779


TypeError: _solve_alpha() takes 3 positional arguments but 4 were given